In [1]:
from pathlib import Path

input_data = Path("../input_data/08").read_text()

In [6]:
default_display_signals = {
    1: {          "c",           "f"     },

    7: {"a",      "c",           "f"     },

    4: {     "b", "c", "d",      "f"     },

    2: {"a",      "c", "d", "e",      "g"},
    3: {"a",      "c", "d",      "f", "g"},

    5: {"a", "b",      "d",      "f", "g"},

    6: {"a", "b",      "d", "e", "f", "g"},
    0: {"a", "b", "c",      "e", "f", "g"},
    9: {"a", "b", "c", "d",      "f", "g"},
    8: {"a", "b", "c", "d", "e", "f", "g"},
}


available_signals = {"a", "b", "c", "d", "e", "f", "g"}

In [ ]:
for line in input_data.split("\n"):
    signal_conversion_options = {sig: available_signals.copy() for sig in available_signals}
    signals, outputs = line.split("|")

    #

In [20]:
lines = [line for line in input_data.split("\n")]

for line in input_data.split("\n"):
    signal_conversion_options = {sig: available_signals.copy() for sig in available_signals}
    signals, outputs = line.split("|")


    for signal in sorted(signals.split(), key=lambda x: len(x)):


        signal_set = set(signal)

        viable_default_signals = [v for v in default_display_signals.values() if len(v) == len(signal_set)]
        print(f"{viable_default_signals=}")
        all_options = set().union(*viable_default_signals)
        for wire in signal:
            signal_conversion_options[wire] &= all_options



        display(signal_conversion_options)



    break




viable_default_signals=[{'c', 'f'}]


{'c': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'a': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'd': {'c', 'f'},
 'f': {'c', 'f'},
 'e': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'b': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'g': {'a', 'b', 'c', 'd', 'e', 'f', 'g'}}

viable_default_signals=[{'c', 'a', 'f'}]


{'c': {'a', 'c', 'f'},
 'a': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'd': {'c', 'f'},
 'f': {'c', 'f'},
 'e': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'b': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'g': {'a', 'b', 'c', 'd', 'e', 'f', 'g'}}

viable_default_signals=[{'c', 'd', 'b', 'f'}]


...

{'c': {'a', 'c', 'f'},
 'a': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'd': {'c', 'f'},
 'f': {'c', 'f'},
 'e': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'b': {'b', 'c', 'd', 'f'},
 'g': {'b', 'c', 'd', 'f'}}

viable_default_signals=[{'c', 'a', 'f', 'e', 'b', 'g'}, {'a', 'd', 'f', 'e', 'b', 'g'}, {'c', 'a', 'd', 'f', 'b', 'g'}]


{'c': {'a', 'c', 'f'},
 'a': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'd': {'c', 'f'},
 'f': {'c', 'f'},
 'e': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'b': {'b', 'c', 'd', 'f'},
 'g': {'b', 'c', 'd', 'f'}}

viable_default_signals=[{'c', 'a', 'd', 'f', 'e', 'b', 'g'}]


{'c': {'a', 'c', 'f'},
 'a': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'd': {'c', 'f'},
 'f': {'c', 'f'},
 'e': {'a', 'b', 'c', 'd', 'e', 'f', 'g'},
 'b': {'b', 'c', 'd', 'f'},
 'g': {'b', 'c', 'd', 'f'}}